# Probabilistic Linear Discriminant Analysis (PLDA)

This notebook illustrate how to use a PLDA with the [beer framework](https://github.com/beer-asr/beer).

In [1]:
%load_ext autoreload
%autoreload 2

# Add the path of the beer source code ot the PYTHONPATH.
import sys
sys.path.insert(0, '../')


import numpy as np
import torch
import torch.optim

from torch import nn
from torch.autograd import Variable


# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
from bokeh.models import LinearAxis, Range1d

# Beer framework
import beer

# Convenience functions for plotting.
import plotting

output_notebook(verbose=False)

Loading BokehJS ...

## Data 

In [2]:
ntargets = 3
N = 200
Xs = []
labels = []
cov = np.array([
    [1, -.7],
    [-.7, 3], 
])

shift = 100
mean = 5 * np.array([1, .25]) + shift
X = np.random.multivariate_normal(mean, cov, size=N)
labels.append(np.ones(len(X)) * 0)
Xs.append(X)

mean = 3 * np.array([0, 0]) + shift
X = np.random.multivariate_normal(mean, cov, size=N)
labels.append(np.ones(len(X)))
Xs.append(X)

mean = 5 * np.array([-1, -.25]) + shift
X = np.random.multivariate_normal(mean, cov, size=N)
labels.append(np.ones(len(X)) * 2)
Xs.append(X)

idxs = np.arange(0, ntargets * N)
np.random.shuffle(idxs)
data = np.vstack(Xs)[idxs]
labels = np.hstack(labels)[idxs]

test_data = data[-100:]
test_labels = labels[-100:]
data = data[:-100]
labels = labels[:-100]


x_range = (shift - 10, shift + 10)
y_range = (shift - 10, shift + 10)

fig = figure(title='Synthetic data', width=400, height=400, x_range=x_range,
             y_range=y_range)
colors = ['salmon', 'blue', 'green', 'yellow', 'black', 'red', 'cyan', 'purple', 'brown', 'pink']
for sX, color in zip(Xs, colors):
    fig.circle(sX[:, 0], sX[:, 1], color=color)
show(fig)

## Model

In [3]:
# We use the global mean/cov. matrix of the data to initialize the mixture.
data_mean = torch.from_numpy(data.mean(axis=0)).float()
data_var = torch.from_numpy(np.var(data, axis=0)).float()

gmm_conf = {
    'type': 'Mixture',
    'prior_strength': 1.,
    'components': {
        'type': 'NormalSet',
        'size': 3,
        'covariance': 'full',
        'shared_covariance': False,
        'prior_strength': 1.,
        'noise_std': 1.
    }
}
        
gmm = beer.create_model(gmm_conf, data_mean, data_var).double()

In [4]:
conf = {
    'type': 'Mixture',
    'prior_strength': 1.,
    'components': {
        'type': 'PLDASet',
        'size': 3,
        'dim_noise_subspace': 2,
        'dim_class_subspace': 2,
        'prior_strength': 1.,
        'noise_std': 1.
    }
}
        
plda = beer.create_model(conf, data_mean, .1 * torch.ones_like(data_var)).double()

## Variational Bayes Training

In [5]:
epochs = 200
lrate = 1.
X = torch.from_numpy(data).double()
targets = torch.from_numpy(labels).long()
optimizer = beer.BayesianModelCoordinateAscentOptimizer(
    gmm.mean_field_groups, 
    lrate=lrate
)

elbos = []
for epoch in range(epochs):
    optimizer.zero_grad()
    elbo = beer.evidence_lower_bound(gmm, X, datasize=len(X), labels=targets)
    elbo.natural_backward()
    optimizer.step()
    
    if epochs > 0:
        elbos.append(float(elbo) / len(X))
    
# Plot the ELBO.
fig = figure(title='ELBO', width=400, height=400, x_axis_label='step',
              y_axis_label='ln p(X)')
fig.line(np.arange(len(elbos)), elbos, color='blue')

show(fig)

## Plotting

In [6]:
fig1 = figure(title='PLDA', width=400, height=400, x_range=x_range, y_range=y_range)
fig1.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig1, gmm, alpha=.5, color='blue')
show(fig1)

In [7]:
conf = {
    'type': 'Mixture',
    'prior_strength': 1.,
    'components': {
        'type': 'PLDASet',
        'gmm': gmm,
        'size': 3,
        'dim_noise_subspace': 2,
        'dim_class_subspace': 2,
        'prior_strength': 1.,
        'noise_std': 1.
        
    }
}
plda = beer.create_model(conf, data_mean.double(), torch.ones_like(data_var).double())

fig1 = figure(title='PLDA', width=400, height=400, x_range=x_range, y_range=y_range)
fig1.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig1, plda, alpha=.5, color='blue')
show(fig1)

In [8]:
epochs = 200
lrate = 1.
X = torch.from_numpy(data).double()
targets = torch.from_numpy(labels).long()
optimizer = beer.BayesianModelCoordinateAscentOptimizer(
    plda.mean_field_groups, 
    lrate=lrate
)

elbos = []
for epoch in range(epochs):
    optimizer.zero_grad()
    elbo = beer.evidence_lower_bound(plda, X, datasize=len(X), labels=targets)
    elbo.natural_backward()
    optimizer.step()
    
    if epochs > 0:
        elbos.append(float(elbo) / len(X))
    
# Plot the ELBO.
fig = figure(title='ELBO', width=400, height=400, x_axis_label='step',
              y_axis_label='ln p(X)')
fig.line(np.arange(len(elbos)), elbos, color='blue')

show(fig)

In [9]:
fig1 = figure(title='PLDA', width=400, height=400, x_range=x_range, y_range=y_range)
fig1.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig1, plda, alpha=.5, color='blue')
show(fig1)